# All in 1

In [30]:
task_id = 233093
Layers = [2]
Heads = [4]
Emedding_dim = 128 #The embedding size is set one by one to avoid the out of memory error
total_combs = len(Layers)*len(Heads)
batch_size = 64


In [32]:
def create_parameters(task_id, Layers, Heads, Emedding_dim, batch_size):
    
    parameters = {
    "task_id" : task_id,
    "Layers": Layers,
    "Heads": Heads,
    "Emedding_dim": Emedding_dim,
    "bath_size": batch_size,
    }

    return parameters

In [22]:
parameters = create_parameters(task_id, Layers, Heads, Emedding_dim)

In [ ]:
''' 
import os

path_project = "/home/diego/Git/thesis-tabtrans/new_folder"

# Use the os.makedirs() function to create the new folder
os.makedirs(path_project) #create new_folder

'''


In [4]:
project_path = "/home/diego/Git/thesis-tabtrans"

In [9]:
import sys
sys.path.append(project_path) #import folders from the project_path

import os
import numpy as np
import torch
import torch.nn as nn
from utils import training, callback, evaluating, attention, data, plots 
from sklearn import datasets, model_selection
import skorch
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import openml
from sklearn import datasets, model_selection
from skorch.callbacks import Checkpoint, EarlyStopping, LoadInitState, EpochScoring, Checkpoint, TrainEndCheckpoint

In [6]:
task_id = 233093
dataset_name = data.get_dataset_name(task_id)

X_train, X_test, y_train, y_test, n_instances, n_labels, n_numerical, n_categories = data.import_data(task_id, "task")

#####################################################
    # Split data
#####################################################

#getting validation indices
train_indices, val_indices = model_selection.train_test_split(np.arange(X_train.shape[0]), test_size=0.333) #1/9 of train is equal to 10% of total




INFO:openml.datasets.dataset:pickle write mfeat-factors


In [7]:
n_heads = 4 # In average 4 works better
embed_dim = 256 # In average 256 works better
n_layers = 3
ff_pw_size = 30  #this value because of the paper 
attn_dropout = 0.3 #paper
ff_dropout = 0.1 #paper value
aggregator = "cls"
aggregator_parameters = None
decoder_hidden_units = [128,64] #paper value
decoder_activation_fn = nn.ReLU()
need_weights = False
numerical_passthrough = False

In [8]:
#module
module = training.build_module(
    n_categories, # List of number of categories
    n_numerical, # Number of numerical features
    n_heads, # Number of heads per layer
    ff_pw_size, # Size of the MLP inside each transformer encoder layer
    n_layers, # Number of transformer encoder layers    
    n_labels, # Number of output neurons
    embed_dim,
    attn_dropout, 
    ff_dropout, 
    aggregator, # The aggregator for output vectors before decoder
    rnn_aggregator_parameters=aggregator_parameters,
    decoder_hidden_units=decoder_hidden_units,
    decoder_activation_fn=decoder_activation_fn,
    need_weights=need_weights,
    numerical_passthrough=numerical_passthrough
)

/home/diego/anaconda3/envs/tabtrans/lib/python3.11/site-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer was not TransformerEncoderLayer
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [13]:
def new_folder(project_path, new_folder_name):
    # Create the full path for the new folder
    new_folder_path = os.path.join(project_path, new_folder_name)

    # Check if the new folder exists
    if not os.path.exists(new_folder_path):
        # If it doesn't exist, create the new folder
        os.makedirs(new_folder_path)

In [17]:
# Define the project path
project_path = "/home/diego/Git/thesis-tabtrans"

# Specify the name of the new folder
new_folder_name = "data"

new_folder(project_path, new_folder_name)

In [23]:
parameters

{'task_id': 233093, 'Layers': [2], 'Heads': [4], 'Emedding_dim': 128}

In [27]:
num = 0
for i in range(2):
    for z in range(3):
        num += 1
        print(num)

1
2
3
4
5
6


In [33]:
def model_creation(parameters, project_path):
    
    #extract parameters
    task_id = parameters["task_id"]
    Layers = parameters["Layers"]
    Heads = parameters["Heads"]
    Emedding_dim = parameters["Emedding_dim"]
    batch_size = parameters["batch_size"]
    total_combs = len(Layers)*len(Heads)

    dataset_name = data.get_dataset_name(task_id)

    X_train, X_test, y_train, y_test, n_instances, n_labels, n_numerical, n_categories = data.import_data(task_id, "task")
    #getting validation indices
    train_indices, val_indices = model_selection.train_test_split(np.arange(X_train.shape[0]), test_size=0.333) #1/9 of train is equal to 10% of total


    #create the folder to save the dataset experiments if it doesn't exist
    new_folder(project_path, "data_models")
    path_of_data_models = os.path.join(project_path, "data_models") #path of the folder data_models

    #create the folder for specific dataset name
    new_folder(path_of_data_models, dataset_name)
    path_of_dataset = os.path.join(path_of_data_models, dataset_name) #path of the folder dataset 

    experiment_num = 1
    
    for layer in layers:
        for head in heads:
            #experiment i folder
            new_folder(path_of_dataset, f"experiment_{experiment_num}")
            path_of_experiment = os.path.join(path_of_dataset, f"experiment_{experiment_i}") #In this folder it will be saved the model and images


            model = skorch.NeuralNetClassifier(
                module=module,
                criterion=torch.nn.CrossEntropyLoss,
                optimizer=torch.optim.AdamW,
                device="cuda" if torch.cuda.is_available() else "cpu",
                batch_size = batch_size,
                max_epochs = 100,
                train_split=skorch.dataset.ValidSplit(((train_indices, val_indices),)),
                callbacks=[
                    ("balanced_accuracy", skorch.callbacks.EpochScoring("balanced_accuracy", lower_is_better=False)),
                    ("duration", skorch.callbacks.EpochTimer()),
                    EpochScoring(scoring='accuracy', name='train_acc', on_train=True),
                    Checkpoint(monitor='valid_acc_best',dirname = path_of_experiment, load_best = True),
                    EarlyStopping(patience=3)

                ],
                optimizer__lr=1e-4,
                optimizer__weight_decay=1e-4
            )

            # Define Checkpoint and TrainEndCheckpoint callbacks with custom directory
            cp = Checkpoint()
            train_end_cp = TrainEndCheckpoint()

            model = model.fit(X={
                    "x_numerical": X_train[:, :n_numerical].astype(np.float32),
                    "x_categorical": X_train[:, n_numerical:].astype(np.int32)
                    }, 
                    y=y_train.astype(np.int64)
                    )
            
            '''
            TO DO LIST:
            - TEST THE MODEL
            - FROM THOSE RESULTS CREATE A PANDAS
            - SAVE THE PLOTS IN AN SPECIFIC FOLDER
            '''

    
